# NER ภาษาไทย

เขียนโดย นาย วรรณพงษ์  ภัททิยไพบูลย์

นักศึกษาชั้นปีที่ 1 สาขาวิทยาการคอมพิวเตอร์และสารสนเทศ คณะวิทยาศาสตร์ประยุกต์และวิศวกรรมศาสตร์
มหาวิทยาลัยขอนแก่น วิทยาเขตหนองคาย

ใช้ข้อมูล Nutcha's theses จาก http://pioneer.chula.ac.th/~awirote/Data-Nutcha.zip

**ตัวอย่างข้อมูล**

ทางด้าน<NEp>นายกมล เจริญเกษ</NEp> ปศุสัตว์<NEl>กำแพงเพชร</NEl> เผยว่า ขณะนี้ทางจังหวัดไม่ต้องประกาศเป็นพื้นที่สีแดง



In [1]:
!wget http://pioneer.chula.ac.th/~awirote/Data-Nutcha.zip

--2018-05-25 07:56:33--  http://pioneer.chula.ac.th/~awirote/Data-Nutcha.zip
Resolving pioneer.chula.ac.th (pioneer.chula.ac.th)... 161.200.192.25
Connecting to pioneer.chula.ac.th (pioneer.chula.ac.th)|161.200.192.25|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1526832 (1.5M) [application/zip]
Saving to: ‘Data-Nutcha.zip.3’

Data-Nutcha.zip.3   100%[===================>]   1.46M   179KB/s    in 9.2s    

2018-05-25 07:56:42 (163 KB/s) - ‘Data-Nutcha.zip.3’ saved [1526832/1526832]



In [0]:
# แตกไฟล์
import zipfile
with zipfile.ZipFile("Data-Nutcha.zip","r") as zip_ref:
    zip_ref.extractall()

In [3]:
!ls

Data	 Data-Nutcha.zip    Data-Nutcha.zip.2  nltk_data
datalab  Data-Nutcha.zip.1  Data-Nutcha.zip.3


In [4]:
!pip3 install  --ignore-installed https://github.com/PyThaiNLP/pythainlp/archive/dev.zip

    100% |████████████████████████████████| 10.6MB 4.9MB/s 
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/dc/83/15f7833b70d3e067ca91467ca245bae0f6fe56ddc7451aa0dc5606b120f2/pytz-2018.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/27/cc/6dd9a3869f15c2edfab863b992838277279ce92663d334df9ecf5106f5c6/idna-2.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/cb/6965947c13a94236f6d4b8223e21beb4d576dc72e8130bd7880f600839b8/urllib3-1.22-py2.py3-none-any.whl
  U

In [0]:
import codecs
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re

In [0]:
pattern = r'\<(.*?)\>(.*?)\<\/(.*?)\>'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด <NEl>ไง</NEl> ให้เป็น ('NEl','ไง','NEl')

In [0]:
def toolner_to_tag(text):
	text=re.sub("\<[A-Z]+\>",'',text.strip())
	text=re.sub("\<\/[A-Z]+\>",'',text)
	text=re.sub("(\<\/NE(.*?)\>)","\\1***",text).replace('<N','***<N')# text.replace('>','>***') # ตัดการกับพวกไม่มี tag word
	text=text.split('***')
	text2=[]
	for i in text:
		if "<NE" in i:
			text2.append(i)
		else:
			text2.append("<word>"+i+"</word>")
	text=re.sub("<word></word>","","".join(text2))
	return text

In [0]:
def text2conll2002(text):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text)
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text)
    j=0
    conll2002=""
    for tagopen,text,tagclose in tag:
        word_cut=word_tokenize(text,engine='newmm')
        i=0
        txt5=""
        while i<len(word_cut):
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
            else:
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
    return postag(conll2002)

In [0]:
# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine='perceptron')
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
        i+=1
    return text

In [0]:
def get_data(fileopen):
	"""
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
	with codecs.open(fileopen, 'r',encoding='utf8') as f:
		lines = f.read().splitlines()
	return lines

In [0]:
def alldata(lists):
    text=""
    for data in lists:
        text+=text2conll2002(data)
        text+='\n'
    return text

In [0]:
def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        txt=text2conll2002(data).split('\n')
        for d in txt:
            tt=d.split('\t')
            if d!="": data_num.append((tt[0],tt[1],tt[2]))
        #print(data_num)
        data_all.append(data_num)
    #print(data_all)
    return data_all

In [0]:
def alldata_list_str(lists):
	string=""
	for data in lists:
		string1=""
		for j in data:
			string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
		string1+="\n"
		string+=string1
	return string

In [0]:
def write_conll2002(file_name,data):
    """
    ใช้สำหรับเขียนไฟล์
    """
    with codecs.open(file_name, "w", "utf-8-sig") as temp:
        temp.write(data)
    return True

In [0]:
class TrainChunker(nltk.ChunkParserI):
    """
    ใช้ในการ Train และรัน
    """
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in sent] for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)#nltk.NaiveBayesClassifier.train(train_data)
        self.tagger = nltk.tag.BigramTagger(train_data, backoff=self.tagger)
        self.tagger = nltk.tag.TrigramTagger(train_data, backoff=self.tagger)
        print(self.tagger.evaluate(train_data))
    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)#classify(pos_tags)#
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word.replace('<space>',' '), pos, chunktag) for ((word,pos),chunktag) in zip(sentence, chunktags)]
        #print(conlltags)
        return conlltags

In [0]:
def run(lists):
    """
    ใช้ในการทดสอบ NER
    """
    data=lists
    tag=TrainChunker(data)
    while True:
        texts=input("Text : ")
        toword=word_tokenize(texts,engine='newmm')
        pos=pos_tag(toword,engine='perceptron')
        ner=tag.parse(pos)
        print([(word, chunktag) for (word,pos,chunktag) in ner])

In [0]:
def get_data_tag(listd):
	list_all=[]
	c=[]
	for i in listd:
		if i !='':
			c.append((i.split("	")[0],i.split("	")[1],i.split("	")[2]))
		else:
			list_all.append(c)
			c=[]
	return list_all

In [18]:
!ls Data\

SSG  Tags Inserted  WSG


In [19]:
!ls Data/

SSG  Tags Inserted  WSG


In [20]:
!ls Data\

SSG  Tags Inserted  WSG


In [21]:
list_file=glob.glob('Data/Tags Inserted/*.txt')
for i in list_file:
  print(i)
  listdata=get_data(i)
  datatofile=alldata_list_str(alldata_list(listdata))
  write_conll2002(i+".tag",datatofile)

Data/Tags Inserted/news_00002-dws.txt
Data/Tags Inserted/news_00007-dws.txt
Data/Tags Inserted/news_00004-dws.txt
Data/Tags Inserted/news_00001-dws.txt
Data/Tags Inserted/news_00005-dws.txt
Data/Tags Inserted/news_00008-dws.txt
Data/Tags Inserted/news_00003-dws.txt
Data/Tags Inserted/news_00006-dws.txt


In [22]:
list_file=glob.glob('Data/Tags Inserted/*.tag')
listalll=[]
for i in list_file:
  print(i)
  listdata=get_data(i)
  datatofile=get_data_tag(listdata)
  listalll.extend(datatofile)

Data/Tags Inserted/news_00005-dws.txt.tag
Data/Tags Inserted/news_00007-dws.txt.tag
Data/Tags Inserted/news_00002-dws.txt.tag
Data/Tags Inserted/news_00006-dws.txt.tag
Data/Tags Inserted/news_00008-dws.txt.tag
Data/Tags Inserted/news_00003-dws.txt.tag
Data/Tags Inserted/news_00001-dws.txt.tag
Data/Tags Inserted/news_00004-dws.txt.tag


In [25]:
tag=TrainChunker(listalll) # Train

0.8886517326656473


In [0]:
def run2(data,tag):
    """
    ใช้ในการทดสอบ NER
    """
    toword=word_tokenize(data,engine='newmm')
    pos=pos_tag(toword,engine='perceptron')
    ner=tag.parse(pos)
    print([(word, chunktag) for (word,pos,chunktag) in ner])

In [27]:
run2("ผมเป็นนักศึกษามหาวิทยาลัยขอนแก่น",tag)

[('ผม', 'O'), ('เป็น', 'O'), ('นักศึกษา', 'O'), ('มหาวิทยาลัยขอนแก่น', 'B-NEl')]


In [28]:
run2("ผมเรียนที่มหาวิทยาลัยขอนแก่นวิทยาเขตหนองคาย",tag)

[('ผม', 'O'), ('เรียน', 'O'), ('ที่', 'O'), ('มหาวิทยาลัยขอนแก่น', 'B-NEl'), ('วิทยาเขต', 'I-NEl'), ('หนองคาย', 'I-NEl')]


In [33]:
run2("ผมเดินทางไปจังหวัดขอนแก่น",tag)

[('ผม', 'O'), ('เดินทาง', 'O'), ('ไป', 'O'), ('จังหวัด', 'O'), ('ขอนแก่น', 'O')]


In [39]:
run2("ประเทศไทยมีนโยบายที่เรียกว่าไทยแลนด์ 4.0",tag)

[('ประเทศไทย', 'B-NElo'), ('มี', 'O'), ('นโยบาย', 'O'), ('ที่', 'O'), ('เรียก', 'O'), ('ว่า', 'O'), ('ไทยแลนด์', 'O'), (' ', 'O'), ('4.0', 'O')]


In [35]:
run2("ประเทศไทยมีประชาธิปไตยมาแล้วกี่ปี",tag)

[('ประเทศไทย', 'B-NElo'), ('มี', 'O'), ('ประชาธิปไตย', 'O'), ('มา', 'O'), ('แล้ว', 'O'), ('กี่', 'O'), ('ปี', 'O')]


In [38]:
run2("ประเทศมาเลเซียเพิ่งมีการเลือกตั้งไป",tag)

[('ประเทศ', 'O'), ('มาเลเซีย', 'O'), ('เพิ่ง', 'O'), ('มี', 'O'), ('การเลือกตั้ง', 'O'), ('ไป', 'O')]


In [0]:
run2("ทางด้านนายกมล เจริญเกษ ปศุสัตว์กำแพงเพชร เผยว่า ขณะนี้ทางจังหวัดไม่ต้องประกาศเป็นพื้นที่สีแดง",tag)

# ทดลอง ?

- ลองเปลี่ยนตัวตัดคำจาก newmm ของ PyThaiNLP เป็นตัวอื่น ๆ เช่น deepcut
สำหรับ PyThaiNLP อ่านเอกสารได้ที่ https://github.com/PyThaiNLP/pythainlp/blob/dev/docs/pythainlp-1-6-thai.md#word_tokenize

- ลอง build ฐานข้อมูล NER ขึ้นมา แล้วปรับแต่ง regex ให้เข้ากับฐานข้อมูล

- โค้ดนี้รองรับการเปลี่ยนตัวตัดคำแต่ยังสามารถใช้ข้อมูลเดิมได้

-  ปรับเป็น NaiveBayesClassifier